In [1]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
from PIL import Image
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
import tempfile
import os


In [3]:

class VideoEditor:
    def __init__(self, fps=30, fourcc='mp4v', frame_size=(640, 480)):
        self.fps = fps
        self.frame_size = frame_size
        self.fourcc = cv2.VideoWriter_fourcc(*fourcc)
        self.temp_video_path = tempfile.mktemp(suffix='.mp4')
        self.video_writer = cv2.VideoWriter(self.temp_video_path, self.fourcc, self.fps, self.frame_size)
        self.current_time = 0  # in seconds
        self.audio_clips = []

    def _resize_frame(self, frame):
        return cv2.resize(frame, self.frame_size)

    def _convert_to_cv2(self, image):
        if isinstance(image, str):
            img = cv2.imread(image)
        elif isinstance(image, Image.Image):
            img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        elif isinstance(image, np.ndarray):
            img = image
        else:
            raise ValueError("Unsupported image type.")
        return self._resize_frame(img)
    
    def add_images_from_list(self, images, duration_sec):
        """
        Adds several images to the video.

        Args:
            images: Can be a string (directory path), a list of strings (image file paths),
                    a list of OpenCV images (numpy arrays), or a list of Pillow images.
            duration_sec: The total duration (in seconds) that these images should occupy in the video.
        """
        image_list = []

        if isinstance(images, str) and os.path.isdir(images):
            for filename in sorted(os.listdir(images)):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    image_list.append(os.path.join(images, filename))
        elif isinstance(images, list):
            if all(isinstance(img, str) for img in images):
                image_list = images
            elif all(isinstance(img, (np.ndarray, Image.Image)) for img in images):
                image_list = images
            else:
                raise ValueError("List must contain only strings (paths), OpenCV images, or Pillow images.")
        else:
            raise ValueError("Unsupported 'images' type. Must be a directory path (string), a list of paths, a list of OpenCV images, or a list of Pillow images.")

        if not image_list:
            print("No images found to add.")
            return

        single_image_duration = duration_sec / len(image_list)

        for img in image_list:
            self.add_image(img, single_image_duration)

    def add_image(self, image, duration_sec):
        """
        Adds an image for a given duration (in seconds).
        """
        frame = self._convert_to_cv2(image)
        frame_count = int(self.fps * duration_sec)
        for _ in range(frame_count):
            self.video_writer.write(frame)
        self.current_time += duration_sec

    def add_video(self, video_path):
        """
        Adds another video (with its original duration).
        """
        cap = cv2.VideoCapture(video_path)
        video_fps = cap.get(cv2.CAP_PROP_FPS)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = self._resize_frame(frame)
            self.video_writer.write(frame)
        duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / video_fps
        cap.release()
        self.current_time += duration

    def add_audio(self, audio_path, start_time=None, end_time=None):
        """
        Adds audio from start_time to end_time (both in seconds).
        """
        audio_clip = AudioFileClip(audio_path)
        if start_time is not None or end_time is not None:
            audio_clip = audio_clip.subclip(start_time or 0, end_time or audio_clip.duration)
        self.audio_clips.append((audio_clip, self.current_time))

    def save(self, output_path):
        """
        Finalizes the video and merges audio if present.
        """
        self.video_writer.release()

        final_clip = VideoFileClip(self.temp_video_path)

        if self.audio_clips:
            all_audios = []
            for audio, offset in self.audio_clips:
                all_audios.append(audio.set_start(offset))
            composite_audio = CompositeAudioClip(all_audios)
            final_clip = final_clip.set_audio(composite_audio)

        final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')
        final_clip.close()
        os.remove(self.temp_video_path)


In [4]:
from PIL import Image

editor = VideoEditor(fps=30, frame_size=(512, 512))

# Add an image from file for 3 seconds
editor.add_images_from_list("generated", duration_sec=30)

# Add a Pillow image
#editor.add_image(Image.new("RGB", (640, 480), color="blue"), duration_sec=20)

# Add an OpenCV image
#import numpy as np
#opencv_image = np.zeros((480, 640, 3), dtype=np.uint8)
#editor.add_image(opencv_image, duration_sec=1)

# Add a video
#editor.add_video("sample_video.mp4")

# Add an audio clip from 0s to 5s, starting at the current video time
#editor.add_audio("background.mp3", start_time=0, end_time=5)

# Save to file
editor.save("final_output.mp4")


Moviepy - Building video final_output.mp4.
Moviepy - Writing video final_output.mp4



Moviepy - Done !
Moviepy - video ready final_output.mp4
